In [1]:
!pip install arxiv

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 5.4 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6048 sha256=15b77d7630519d493b62f7230582adc1ea257f07e5be02973a05ad7dd3f67f10
  Stored in directory: /home/nitish/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k


In [3]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [20]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)

wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [21]:
wiki.name

'wikipedia'

In [13]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader=WebBaseLoader('https://docs.smith.langchain.com')

docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())

retriever=vectordb.as_retriever()

retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d54a58632e0>)

In [15]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,'langsmith_search','Search for the information about LangSmith. For any questions about LangSmith')

In [16]:
retriever_tool.name

'langsmith_search'

In [19]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)

arxiv.name


'arxiv'

In [23]:
tools=[wiki,arxiv,retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/nitish/Documents/ML_Projects/LangChain/.venv/lib/python3.10/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for the information about LangSmith. For any questions about LangSmith', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7d54d661e560>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectors

In [24]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')

from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model='gpt-3.5-turbo-0125',temperature=0)


In [26]:
from langchain import hub
prompt=hub.pull('hwchase17/openai-functions-agent')
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

### Agents

In [27]:
from langchain.agents import create_openai_tools_agent

agent=create_openai_tools_agent(llm,tools,prompt)

In [29]:
## Agent Executer

from langchain.agents import AgentExecutor


agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

agent_executor


AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [30]:
agent_executor.invoke({'input':'Tell me about Langsmith'})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We provide 

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith works independently and does not require the use of LangChain. Here are the steps to get started with LangSmith:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on Create API Key.\n\n3. Set up your environment:\n   - Set environment variables:\n     ```\n     export LANGCHAIN_TRACING_V2=true\n     export LANGCHAIN_API_KEY=<your-api-key>\n     export OPENAI_API_KEY=<your-openai-api-key>\n     ```\n\n4. Log your first trace:\n   - LangSmith provides multiple ways to log traces. You can refer to the provided code examples for logging your first trace.\n\nFor more detailed information and tutori